# Analyzing CIA Factbook Data Using SQL
In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

In [2]:
%%sql
SELECT * FROM sqlite_master 
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT * FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

In [4]:
%%sql
SELECT MIN(population) min_ppl, 
        MAX(population) max_ppl, 
        MIN(population_growth) min_ppl_growth, 
        MAX(population_growth) max_ppl_growth
FROM facts;

Done.


min_ppl,max_ppl,min_ppl_growth,max_ppl_growth
0,7256490011,0.0,4.02


## Exploring Outliers

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population) 
                    FROM facts) OR
        population = (SELECT MAX(population) 
                      FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Exploring Average Population and Area

In [6]:
%%sql
SELECT ROUND(AVG(population)) avg_ppl, 
       ROUND(AVG(area)) avg_area
FROM facts;

Done.


avg_ppl,avg_area
62094928.0,555094.0


## Finding Densely Poplulated Countries

In [7]:
%%sql 
SELECT * FROM facts
WHERE population > (SELECT AVG(population) 
                    FROM facts) AND
      area > (SELECT AVG(area) 
              FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
40,cg,"Congo, Democratic Republic of the",2344858,2267048,77810,79375136,2.45,34.88,10.07,0.27
53,eg,Egypt,1001450,995450,6000,88487396,1.79,22.9,4.77,0.19
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22
61,fr,France,643801,640427,3374,66553766,0.43,12.38,9.16,1.09
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16
79,ir,Iran,1648195,1531595,116600,81824270,1.2,17.99,5.94,0.07
114,mx,Mexico,1964375,1943945,20430,121736809,1.18,18.78,5.26,1.68


## Exploring Countries with the Highest Ratios of Water to Land

In [8]:
%%sql
SELECT name country, 
       CAST(area_water as Float)/CAST(area_land as Float) ratio_water_to_land
FROM facts
ORDER BY 2 DESC
LIMIT 10

Done.


country,ratio_water_to_land
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


## Finding Countries with Higher Death Rate than Birth Rate

In [9]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE death_rate > birth_rate

Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16
